In [12]:
!pip install praw
import praw
import pandas as pd
import datetime as dt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from bs4 import BeautifulSoup

You should consider upgrading via the 'c:\users\aditya kumar\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to C:\Users\aditya
[nltk_data]     kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Setting up Reddit AuthO

In [13]:
reddit = praw.Reddit(client_id='WK7Bb4kSnRez8g',
                     client_secret='NYWBOS04ouTSPO0cKhsMLqs-SWQ',
                     user_agent='Reddit_India_Data')
india_subreddit = reddit.subreddit('india')

In [14]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]
post_dict = []

In [15]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def readableTime(created):
    return dt.datetime.utcfromtimestamp(created)


#To convert data elements into string objects
def toStr(item):
    return str(item)

_replace = re.compile('[/(){}\|@,;]')
badSymbols = re.compile('[^0-9a-z #+_]')
stopWords = set(stopwords.words('english'))

#To clean text within dataset

# sentences = 
lemmatizer = WordNetLemmatizer()
def cleanText(string):
    string = BeautifulSoup(string, "lxml").string
    string = string.lower() 
    string = _replace.sub(' ', string)
    string = badSymbols.sub('', string)
    string=nltk.sent_tokenize(string)
    words = [lemmatizer.lemmatize(word) for word in string if word not in set(stopwords.words('english'))]
    string= ' '.join(words)  
#     string = ' '.join(word for word in string.split() if word not in stopWords)
    return string

In [16]:
for flair in flairs:
    submissions = india_subreddit.search(f"flair_name:{flair}",limit=500)
    for submission in submissions:
        post_dict.append([submission.title,
                submission.id,
                submission.link_flair_text,
                submission.author,
                submission.url,
                submission.num_comments,
                submission.score,
                submission.created])
post_dict = pd.DataFrame(post_dict,columns=['title', 'id', 'flair', 'author', 'url', 'num_comments', 'score', 'created'])
topics = pd.DataFrame(post_dict)
#Sorting data based in ascending order of flair
topics.sort_values('flair', inplace=True)
#Changing Unix time to UTC time.
topics['created'] = topics['created'].apply(readableTime)
# del topics['created']
#Deleting any duplicate rows
topics.drop_duplicates(subset=None, inplace=True)
topics.to_csv('Data.csv', index=False)

# DB Cleaning and Text Pre Processing

In [25]:
df = pd.read_csv('Data.csv')
df.head()
df['title'] =df['title'].apply(toStr)
df['title'] =df['title'].apply(cleanText)
df['num_comments'] =df['num_comments'].apply(toStr)
df['num_comments'] =df['num_comments'].apply(cleanText)
df['author'] =df['author'].apply(toStr)
df['author'] =df['author'].apply(cleanText)
# feature_Grp = df['title'] + df['url']
# df['feature_Grp'] = feature_Grp
df.to_csv('Final_formatted_data.csv', index=False)

In [26]:
df = pd.read_csv('Final_formatted_data.csv')


In [27]:
df.head()

,title,id,flair,author,url,num_comments,score,created
0,hi im anish majumdar author of the isolation...,2059oa,AMA,anish_majumdar,https://www.reddit.com/r/india/comments/2059oa...,36,67,2014-03-11 15:53:58
1,np i am a dentist ama,4giohy,AMA,periomate,https://www.reddit.com/r/india/comments/4giohy...,297,95,2016-04-26 12:58:58
2,hi reddit im siddharth agarwal im currently ...,4saxeh,AMA,iamasid,https://www.reddit.com/r/india/comments/4saxeh...,147,340,2016-07-11 13:32:51
3,ama hi r india im a ca and would love to answe...,5tcgn5,AMA,cap_nemo_1984,https://www.reddit.com/r/india/comments/5tcgn5...,206,77,2017-02-11 05:43:55
4,hey reddit i am sandeep goenka cofounder of z...,5e3mni,AMA,goenkasandeep,https://www.reddit.com/r/india/comments/5e3mni...,218,100,2016-11-21 11:44:31
